In [5]:
from scipy.stats import gamma
import pandas as pd
import numpy as np

from pyscripts.test_utils import rand_edges
from pyscripts.planner import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
schedule = pd.read_csv("schedule.csv", index_col=0)\
    .rename(columns={
        "start": "dep_node", 
        "stop": "arr_node", 
        "departure_time": 
        "dep_time", "arrival_time": 
        "arr_time",
        "day_1": "dep_day",
        "day_2": "arr_day"
    })
schedule.head()

In [ ]:
small_schedule = schedule[
    (schedule.dep_day == schedule.arr_day) & 
    (schedule.arr_day == 0) &
    
]
print(small_schedule.shape)
small_schedule.head()

In [ ]:
rand_edges(0.1, 0.2)

In [ ]:
from scipy.stats import gamma
mean = small_schedule.iloc[0].mean_delay
std = small_schedule.iloc[0].std_delay

alpha = (mean/std)**2
scale = std**2 / mean
gam = gamma(a=alpha, scale=scale)

import matplotlib.pylab as plt
% matplotlib inline

x=np.linspace(-10,600,1000)
plt.grid()
plt.plot(x,gam.cdf(x), label='Gamma distribution')

### Journey Planner 

hypotheses:

    i) given the arcs A->B, if ta_dep < tb_dep => ta_arr < tb_arr (before you depart before you arrive)
    ii) delays distribute with lognorm


simplifications:
    - the number of changes is not taken into account (one would prefer also to take do few stops)


In [53]:
planner = Planner(rand_edges(3, 3), reverse=False)
planner = Planner(rand_edges(2, 2), reverse=False)
planner = Planner(rand_edges(), reverse=False)

departure_node = "name1"
arrival_node = "name5"
time = 10
treshold = 0.75

last_id = planner.compute_plan(start_node, target_node, start_time, treshold)

print("Path:", last_id)
planner.edges

Path: [6, 9, 5]


,dep_name,arr_name,dep_time,arr_time,trip_id,prob,label,mean_delay,std_delay,prev_edge
0,name1,name2,0.5,2.0,-2,NaN,Unvisited,1.000000e-10,1.000000e-10,NaN
1,name1,name2,0.1,1.5,1,1.0,Expanded,1.000000e-10,1.000000e-10,NaN
2,name2,name4,3.0,4.0,4,NaN,Unvisited,1.000000e-10,1.000000e-10,NaN
3,name2,name4,2.0,3.0,3,1.0,Expanded,1.000000e-10,1.000000e-10,1.0
4,name4,name5,5.0,7.0,6,NaN,Unvisited,1.000000e-10,1.000000e-10,NaN
5,name4,name5,4.0,6.0,5,1.0,Visited,1.000000e-10,1.000000e-10,9.0
6,name1,name3,0.5,1.0,7,1.0,Expanded,1.000000e-10,1.000000e-10,NaN
7,name3,name5,5.0,8.0,8,1.0,Visited,1.000000e-10,1.000000e-10,6.0
8,name3,name5,6.0,9.0,9,NaN,Unvisited,1.000000e-10,1.000000e-10,NaN
9,name3,name4,2.5,2.5,10,1.0,Expanded,1.000000e-10,1.000000e-10,6.0


## Test Planner

- `initialize_from`

In [45]:
# direct search
planner = Planner(rand_edges(), reverse=False)

sol = [1, 6]
stop_name = "name1"
time = 0.0

ids = planner.initialize_from(stop_name, time)
display(planner.edges.loc[sol])
assert set(sol) == set(ids)

,dep_name,arr_name,dep_time,arr_time,trip_id,prob,label,mean_delay,std_delay,prev_edge
1,name1,name2,0.1,1.5,1,1.0,Visited,1.000000e-10,1.000000e-10,NaN
6,name1,name3,0.5,1.0,7,1.0,Visited,1.000000e-10,1.000000e-10,NaN


In [46]:
# reverse search
planner = Planner(rand_edges(), reverse=True)

sol = [8, 4]
planner.clear()
stop_name = "name5"
time = 10

ids = planner.initialize_from(stop_name, time)
display(planner.edges.loc[sol])
assert set(sol) == set(ids)

,dep_name,arr_name,dep_time,arr_time,trip_id,prob,label,mean_delay,std_delay,prev_edge
8,name3,name5,6.0,9.0,9,1.0,Visited,1.000000e-10,1.000000e-10,NaN
4,name4,name5,5.0,7.0,6,1.0,Visited,1.000000e-10,1.000000e-10,NaN


- `expand_edge`

In [9]:
# direct search
planner = Planner(rand_edges(0.3, 0.2), reverse=False)
planner.edges.dep_name = "name2" # just to increase the possible choices

planner.edges.loc[1, "prob"] = 1
ids = planner.expand_edge(1) # expand the edge with id 1

sol = [2, 5]
assert set(sol) == set(ids)

planner.edges.sort_values("arr_name")

,dep_name,arr_name,dep_time,arr_time,trip_id,prob,label,mean_delay,std_delay,prev_edge
0,name2,name2,0.5,2.0,-2,NaN,Unvisited,0.2,0.3,NaN
1,name2,name2,0.1,1.5,1,1.000000,Expanded,0.2,0.3,NaN
6,name2,name3,0.5,1.0,7,NaN,Unvisited,0.2,0.3,NaN
2,name2,name4,3.0,4.0,4,0.809090,Visited,0.2,0.3,1.0
3,name2,name4,2.0,3.0,3,NaN,Unvisited,0.2,0.3,NaN
9,name2,name4,2.5,2.5,10,NaN,Unvisited,0.2,0.3,NaN
4,name2,name5,5.0,7.0,6,NaN,Unvisited,0.2,0.3,NaN
5,name2,name5,4.0,6.0,5,0.997251,Visited,0.2,0.3,1.0
7,name2,name5,5.0,8.0,8,NaN,Unvisited,0.2,0.3,NaN
8,name2,name5,6.0,9.0,9,NaN,Unvisited,0.2,0.3,NaN


In [12]:
# reverse search
planner = Planner(rand_edges(0.3, 0.2), reverse=True)
planner.edges.arr_name = "name4" # just to increase the possible choices

planner.edges.loc[4, "prob"] = 1
ids = planner.expand_edge(4) # expand the edge with id 4

sol = [0, 3, 9]
assert set(sol) == set(ids)

planner.edges.sort_values("dep_name")

,dep_name,arr_name,dep_time,arr_time,trip_id,prob,label,mean_delay,std_delay,prev_edge
0,name1,name4,0.5,2.0,-2,0.999700,Visited,0.2,0.3,4.0
1,name1,name4,0.1,1.5,1,NaN,Unvisited,0.2,0.3,NaN
6,name1,name4,0.5,1.0,7,NaN,Unvisited,0.2,0.3,NaN
2,name2,name4,3.0,4.0,4,NaN,Unvisited,0.2,0.3,NaN
3,name2,name4,2.0,3.0,3,0.974961,Visited,0.2,0.3,4.0
7,name3,name4,5.0,8.0,8,NaN,Unvisited,0.2,0.3,NaN
8,name3,name4,6.0,9.0,9,NaN,Unvisited,0.2,0.3,NaN
9,name3,name4,2.5,2.5,10,0.997251,Visited,0.2,0.3,4.0
4,name4,name4,5.0,7.0,6,1.000000,Expanded,0.2,0.3,NaN
5,name4,name4,4.0,6.0,5,NaN,Unvisited,0.2,0.3,NaN


- `best_node`

In [18]:
# direct search
planner = Planner(rand_edges(0.3, 0.2), reverse=False)
visited = np.array([0, 3, 5, 8])

planner.edges.loc[visited, "label"] = Status.Visited

best = planner.best_node(visited)
sol = 0
assert sol == best

planner.edges[planner.edges.label == Status.Visited]

,dep_name,arr_name,dep_time,arr_time,trip_id,prob,label,mean_delay,std_delay,prev_edge
0,name1,name2,0.5,2.0,-2,NaN,Visited,0.2,0.3,NaN
3,name2,name4,2.0,3.0,3,NaN,Visited,0.2,0.3,NaN
5,name4,name5,4.0,6.0,5,NaN,Visited,0.2,0.3,NaN
8,name3,name5,6.0,9.0,9,NaN,Visited,0.2,0.3,NaN


In [19]:
# reverse search
planner = Planner(rand_edges(0.3, 0.2), reverse=True)
visited = np.array([0, 3, 5, 8])

planner.edges.loc[visited, "label"] = Status.Visited

best = planner.best_node(visited)
sol = 8
assert sol == best

planner.edges[planner.edges.label == Status.Visited]

,dep_name,arr_name,dep_time,arr_time,trip_id,prob,label,mean_delay,std_delay,prev_edge
0,name1,name2,0.5,2.0,-2,NaN,Visited,0.2,0.3,NaN
3,name2,name4,2.0,3.0,3,NaN,Visited,0.2,0.3,NaN
5,name4,name5,4.0,6.0,5,NaN,Visited,0.2,0.3,NaN
8,name3,name5,6.0,9.0,9,NaN,Visited,0.2,0.3,NaN


- `done`

In [31]:
# direct search
planner = Planner(rand_edges(0.3, 0.2), reverse=False)

assert planner.done(2, "name4")
assert not planner.done(2, "name1")
assert planner.done(6, "name3")
assert not planner.done(8, "name1")

# reverse search
planner.reverse = True
assert planner.done(2, "name2")
assert not planner.done(2, "name1")
assert planner.done(6, "name1")
assert not planner.done(8, "name4")

planner.edges

,dep_name,arr_name,dep_time,arr_time,trip_id,prob,label,mean_delay,std_delay,prev_edge
0,name1,name2,0.5,2.0,-2,NaN,Unvisited,0.2,0.3,NaN
1,name1,name2,0.1,1.5,1,NaN,Unvisited,0.2,0.3,NaN
2,name2,name4,3.0,4.0,4,NaN,Unvisited,0.2,0.3,NaN
3,name2,name4,2.0,3.0,3,NaN,Unvisited,0.2,0.3,NaN
4,name4,name5,5.0,7.0,6,NaN,Unvisited,0.2,0.3,NaN
5,name4,name5,4.0,6.0,5,NaN,Unvisited,0.2,0.3,NaN
6,name1,name3,0.5,1.0,7,NaN,Unvisited,0.2,0.3,NaN
7,name3,name5,5.0,8.0,8,NaN,Unvisited,0.2,0.3,NaN
8,name3,name5,6.0,9.0,9,NaN,Unvisited,0.2,0.3,NaN
9,name3,name4,2.5,2.5,10,NaN,Unvisited,0.2,0.3,NaN


- `compute_plan`

In [49]:
trials = [
    (Planner(rand_edges(3, 3), reverse=False), [6, 8]),
    (Planner(rand_edges(2, 2), reverse=False), [6, 7]),
    (Planner(rand_edges(), reverse=False), [6, 9, 5])    
]

start_node = "name1"
target_node = "name5"

treshold = 0.75

In [51]:
# direct search
start_time = 0.0

for trial in trials:
    planner = trial[0]
    sol = trial[1]
    
    planner.reverse = False
    path = planner.compute_plan(start_node, target_node, start_time, treshold)
    
    assert path==sol, "Found path: " + str(path) + ". Expected path: " + str(sol)

In [50]:
# reverse search
for trial in trials:
    planner = trial[0]
    sol = trial[1]
    
    planner.reverse = True
    
    start_time = planner.edges.loc[sol[-1]].arr_time # make it start at the arrival time of the solution path, we should find the same exact solution
    
    path = planner.compute_plan(start_node, target_node, start_time, treshold)
    
    assert path==sol, "Found path: " + str(path) + ". Expected path: " + str(sol)